# Projet g5 -- nettoyage - préparation des données

In [1]:
# librairies utiles

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize

In [346]:
# importer les fichiers brut dans des dataFrame

In [374]:
vehicules= pd.read_csv('data/vehicules.csv',dtype=str)
usagers=pd.read_csv('data/usagers.csv',dtype=str)
crtrtic=pd.read_csv('data/caracteristiques.csv',dtype=str)
lieux=pd.read_csv('data/lieux.csv',dtype=str)

In [351]:
# definition type des variables
vehicules['catv']=vehicules['catv'].astype('category')


In [352]:
vehicules.shape
vehicules.info()
vehicules.tail(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1635811 entries, 0 to 1635810
Data columns (total 4 columns):
 #   Column   Non-Null Count    Dtype   
---  ------   --------------    -----   
 0   Num_Acc  1635811 non-null  object  
 1   catv     1635811 non-null  category
 2   num_veh  1635811 non-null  object  
 3   year     1635811 non-null  object  
dtypes: category(1), object(3)
memory usage: 39.0+ MB


,Num_Acc,catv,num_veh,year
1635801,201800057778,07,B01,2018
1635802,201800057778,07,A01,2018
1635803,201800057779,30,A01,2018
1635804,201800057780,30,A01,2018
1635805,201800057780,07,B01,2018
1635806,201800057781,07,A01,2018
1635807,201800057782,07,A01,2018
1635808,201800057782,31,B01,2018
1635809,201800057783,10,B01,2018
1635810,201800057783,10,A01,2018


In [353]:
#vehicules.describe()


vehicules=vehicules.loc[:,['Num_Acc','catv','num_veh','year']]
pd.isnull(vehicules["catv"]).sum()


0

In [354]:
vehicules.head()


,Num_Acc,catv,num_veh,year
0,200500000001,07,A01,2005
1,200500000001,07,B02,2005
2,200500000002,07,A01,2005
3,200500000002,02,B02,2005
4,200500000003,02,A01,2005


In [357]:


print(vehicules["catv"].isna().sum())

0


In [375]:
# FICHIER USAGERS
#usagers.shape
usagers=usagers.drop(['locp','actp','etatp'],axis=1)

In [376]:
usagers[['place','catu','grav','sexe','trajet','secu']]=usagers[['place','catu','grav','sexe','trajet','secu']].astype('category')
usagers['an_nais']=usagers['an_nais'].astype('float64')


In [377]:
usagers.head()
usagers.info()
usagers.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142195 entries, 0 to 2142194
Data columns (total 10 columns):
 #   Column   Dtype   
---  ------   -----   
 0   Num_Acc  object  
 1   place    category
 2   catu     category
 3   grav     category
 4   sexe     category
 5   trajet   category
 6   secu     category
 7   an_nais  float64 
 8   num_veh  object  
 9   year     object  
dtypes: category(6), float64(1), object(3)
memory usage: 77.6+ MB


(2142195, 10)

In [378]:
usagers.loc[:,'grav'].value_counts().sum()

2142195

In [379]:
usagers.isna().sum()*100/len(usagers)

Num_Acc    0.000000
place      5.747749
catu       0.000000
grav       0.000000
sexe       0.000000
trajet     0.023060
secu       2.639209
an_nais    0.112875
num_veh    0.000000
year       0.000000
dtype: float64

In [380]:
usagers.loc[:,'an_nais'].describe()

count    2.139777e+06
mean     1.973446e+03
std      1.852290e+01
min      1.896000e+03
25%      1.961000e+03
50%      1.976000e+03
75%      1.987000e+03
max      2.018000e+03
Name: an_nais, dtype: float64

In [381]:
an_miss=usagers[usagers.loc[:,'an_nais'].isna()]

In [382]:
an_miss.head(10)

,Num_Acc,place,catu,grav,sexe,trajet,secu,an_nais,num_veh,year
786,200500000333,1,1,1,1,5,11,NaN,A01,2005
787,200500000333,1,1,4,1,5,21,NaN,B02,2005
5749,200500002385,1,1,1,1,1,13,NaN,B01,2005
5812,200500002415,1,1,1,2,5,13,NaN,B01,2005
5813,200500002416,1,1,1,1,0,13,NaN,A01,2005
5857,200500002437,1,1,3,1,5,11,NaN,B01,2005
5866,200500002442,1,1,1,1,5,11,NaN,A01,2005
5881,200500002448,1,1,1,1,5,11,NaN,A01,2005
5905,200500002460,1,1,1,1,0,23,NaN,A01,2005
5915,200500002464,2,2,1,2,0,11,NaN,C01,2005


In [383]:
# Remplacer les valeurs manquantes de age par la moyenne
usagers.loc[:,'an_nais'].mean()
usagers['an_nais'].fillna(usagers['an_nais'].mean(), inplace=True)

usagers.isna().sum()*100/len(usagers)

Num_Acc    0.000000
place      5.747749
catu       0.000000
grav       0.000000
sexe       0.000000
trajet     0.023060
secu       2.639209
an_nais    0.000000
num_veh    0.000000
year       0.000000
dtype: float64

In [384]:
pd.crosstab(usagers["grav"],usagers["sexe"],margins=True)

sexe,1,2,All
grav,,,
1,628356,247036,875392
2,43898,13683,57581
3,303247,141584,444831
4,463817,300574,764391
All,1439318,702877,2142195


In [385]:
#pd.crosstab(usagers["place"],vehicules["catv"],margins=True)

In [386]:
# 2% de la variable secu est mqt ---> on peut envisager une imputation par le mode
usagers.loc[:,'secu'].value_counts()

11    1197467
21     389504
13     166913
93     105121
00      68374
23      40865
12      36315
22      22374
31      14976
3        7707
92       7693
91       7653
10       5631
1        3567
2        2669
41       2532
43       2004
42       1856
33        837
32        761
20        754
90         73
30          9
40          2
01          1
Name: secu, dtype: int64

In [387]:
# calcul du mode 
usagers.loc[:,'secu'].mode()

0    11
Name: secu, dtype: category
Categories (25, object): ['00', '01', '1', '10', ..., '90', '91', '92', '93']

In [388]:
# 5% de la variable place est mqt --->  imputation par le mode
usagers.loc[:,'place'].value_counts()

1    1596108
2     225920
0      60766
3      50828
4      43606
5      25069
8       5995
7       4692
9       4078
6       2005
Name: place, dtype: int64

In [389]:
usagers.loc[:,'place'].mode()

0    1
Name: place, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [393]:
# Imputation secu & place
usagers['secu'].fillna(usagers['secu'].mode()[0], inplace=True)
usagers['place'].fillna(usagers['place'].mode()[0], inplace=True)

In [394]:
usagers.isna().sum()*100/len(usagers)

Num_Acc    0.00000
place      0.00000
catu       0.00000
grav       0.00000
sexe       0.00000
trajet     0.02306
secu       0.00000
an_nais    0.00000
num_veh    0.00000
year       0.00000
dtype: float64

In [395]:
# Personnes DCD
usagers['dc']=0

In [396]:
usagers.loc[usagers['grav']==2,'dc']=1


In [397]:
pd.crosstab(usagers["dc"],usagers["secu"],margins=True)

secu,00,01,1,10,11,12,13,2,20,21,...,33,40,41,42,43,90,91,92,93,All
dc,,,,,,,,,,,,,,,,,,,,,
0,68374,1,3567,5631,1254004,36315,166913,2669,754,389504,...,837,2,2532,1856,2004,73,7653,7693,105121,2142195
All,68374,1,3567,5631,1254004,36315,166913,2669,754,389504,...,837,2,2532,1856,2004,73,7653,7693,105121,2142195


In [398]:
usagers.head()

,Num_Acc,place,catu,grav,sexe,trajet,secu,an_nais,num_veh,year,dc
0,200500000001,1,1,4,1,1,11,1976.0,A01,2005,0
1,200500000001,1,1,3,2,3,11,1968.0,B02,2005,0
2,200500000001,2,2,1,1,0,11,1964.0,B02,2005,0
3,200500000001,4,2,1,1,0,31,2004.0,B02,2005,0
4,200500000001,5,2,1,1,0,11,1998.0,B02,2005,0


In [399]:
lieux.head()
lieux.info()
lieux.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958469 entries, 0 to 958468
Data columns (total 19 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Num_Acc  958469 non-null  object
 1   catr     958468 non-null  object
 2   voie     869558 non-null  object
 3   v1       333391 non-null  object
 4   v2       39348 non-null   object
 5   circ     956895 non-null  object
 6   nbv      955738 non-null  object
 7   pr       482985 non-null  object
 8   pr1      481166 non-null  object
 9   vosp     955708 non-null  object
 10  prof     956520 non-null  object
 11  plan     956188 non-null  object
 12  lartpc   902271 non-null  object
 13  larrout  904096 non-null  object
 14  surf     956545 non-null  object
 15  infra    953061 non-null  object
 16  situ     953499 non-null  object
 17  env1     953029 non-null  object
 18  year     958469 non-null  object
dtypes: object(19)
memory usage: 138.9+ MB


(958469, 19)

In [400]:
lieux[['catr','circ','surf','situ']]=lieux[['catr','circ','surf','situ']].astype('category')


In [401]:
lieux=lieux[['Num_Acc','catr','circ','surf','situ','year']]
lieux.head()

,Num_Acc,catr,circ,surf,situ,year
0,200500000001,3,2,1,1,2005
1,200500000002,2,0,1,5,2005
2,200500000003,2,0,2,5,2005
3,200500000004,3,2,1,1,2005
4,200500000005,3,2,2,3,2005


In [402]:
# % de valeurs manquantes
lieux.isna().sum()*100/len(lieux)


Num_Acc    0.000000
catr       0.000104
circ       0.164220
surf       0.200737
situ       0.518535
year       0.000000
dtype: float64

In [403]:
# Caractéristiques

In [404]:
crtrtic.shape
crtrtic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929503 entries, 0 to 929502
Data columns (total 17 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Num_Acc  884060 non-null  object
 1   an       884060 non-null  object
 2   mois     884060 non-null  object
 3   jour     884060 non-null  object
 4   hrmn     884060 non-null  object
 5   lum      884060 non-null  object
 6   agg      884060 non-null  object
 7   int      884060 non-null  object
 8   atm      883994 non-null  object
 9   col      884041 non-null  object
 10  com      884058 non-null  object
 11  adr      755019 non-null  object
 12  gps      446447 non-null  object
 13  lat      437904 non-null  object
 14  long     437899 non-null  object
 15  dep      884057 non-null  object
 16  year     929503 non-null  object
dtypes: object(17)
memory usage: 120.6+ MB


In [405]:
crtrtic[['lum','agg','atm']]=crtrtic[['lum','agg','atm']].astype('category')


crtrtic.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,year
0,200500000001,5,1,12,1900,3,2,1,1,3,11,CD41B,M,5051500,294400,590,2005
1,200500000002,5,1,21,1600,1,2,1,1,1,51,rue de Lille,M,5053700,280200,590,2005
2,200500000003,5,1,21,1845,3,1,1,2,1,51,NaN,M,5054600,280000,590,2005
3,200500000004,5,1,4,1615,1,1,1,1,5,82,NaN,M,5098700,240800,590,2005
4,200500000005,5,1,10,1945,3,1,1,3,6,478,NaN,M,5096400,247500,590,2005


In [406]:
crtrtic=crtrtic.drop(['int','col'],axis=1)

In [407]:
crtrtic.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,atm,com,adr,gps,lat,long,dep,year
0,200500000001,5,1,12,1900,3,2,1,11,CD41B,M,5051500,294400,590,2005
1,200500000002,5,1,21,1600,1,2,1,51,rue de Lille,M,5053700,280200,590,2005
2,200500000003,5,1,21,1845,3,1,2,51,NaN,M,5054600,280000,590,2005
3,200500000004,5,1,4,1615,1,1,1,82,NaN,M,5098700,240800,590,2005
4,200500000005,5,1,10,1945,3,1,3,478,NaN,M,5096400,247500,590,2005


In [408]:
crtrtic.shape

(929503, 15)

In [409]:
crtrtic.isna().sum()*100/len(crtrtic)

Num_Acc     4.888957
an          4.888957
mois        4.888957
jour        4.888957
hrmn        4.888957
lum         4.888957
agg         4.888957
atm         4.896057
com         4.889172
adr        18.771752
gps        51.969278
lat        52.888372
long       52.888909
dep         4.889280
year        0.000000
dtype: float64

In [410]:
# identification des lignes avec 4,88 % de missing sur tte les variables
num_miss=crtrtic[crtrtic.loc[:,'Num_Acc'].isna()]

In [411]:
num_miss.tail(10)  # 45443 valeur manquante sur ttes les colonnes, pas d'imputation possible --> supprimer ?


,Num_Acc,an,mois,jour,hrmn,lum,agg,atm,com,adr,gps,lat,long,dep,year
376069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
376070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
376071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
376072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
376073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
376074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
376075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
376076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
376077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
376078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009


In [412]:
# suppression de 45443 obs avec Valeurs mqt partout
crtrtic=crtrtic.loc[crtrtic.loc[:,'Num_Acc'].notna(),:]


In [413]:
crtrtic.shape

#crtrtic.isna().sum()*100/len(crtrtic)

(884060, 15)

In [414]:
# exploration latitude/longitude mqt 
lat_miss=crtrtic[crtrtic.loc[:,'lat'].isna()]

In [415]:
# au vu du taux élevé de DM pour lat/long/gps 47 % il vaut mieux ne pas inclure ces variables
lat_miss.shape
lat_miss.head(10)

,Num_Acc,an,mois,jour,hrmn,lum,agg,atm,com,adr,gps,lat,long,dep,year
31,200500000032,5,1,18,900,1,2,3,770,NaN,NaN,NaN,NaN,620,2005
42,200500000043,5,1,21,700,3,1,1,423,NaN,NaN,NaN,NaN,620,2005
105,200500000106,5,1,9,230,5,1,1,143,NaN,NaN,NaN,NaN,600,2005
135,200500000136,5,1,12,2145,5,2,1,612,RUE DE MEAUX,NaN,NaN,NaN,600,2005
136,200500000137,5,1,14,745,2,2,2,138,ROUTE DE COMPIEGNE,NaN,NaN,NaN,600,2005
170,200500000171,5,1,15,1345,1,1,1,167,NaN,M,NaN,NaN,760,2005
331,200500000332,5,1,18,1800,5,2,2,601,Rue leprovost,NaN,NaN,NaN,500,2005
332,200500000333,5,1,21,1630,1,1,1,444,NaN,NaN,NaN,NaN,500,2005
382,200500000383,5,1,22,1630,3,1,1,61,NaN,M,NaN,NaN,440,2005
645,200500000646,5,1,18,1245,1,1,8,161,NaN,M,NaN,NaN,790,2005


In [416]:
# construction de la colonne commune [0:1]
print(crtrtic['com'].head())
print(crtrtic['dep'][0:3].head())

0     11
1     51
2     51
3     82
4    478
Name: com, dtype: object
0    590
1    590
2    590
Name: dep, dtype: object


In [417]:
#crtrtic['dep']=crtrtic['dep'].astype(str)
#crtrtic['com']=crtrtic['com'].astype(str)
crtrtic['cp']=crtrtic['dep'].str[0:2] + crtrtic['com']


In [418]:
crtrtic.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,atm,com,adr,gps,lat,long,dep,year,cp
0,200500000001,5,1,12,1900,3,2,1,11,CD41B,M,5051500,294400,590,2005,5911
1,200500000002,5,1,21,1600,1,2,1,51,rue de Lille,M,5053700,280200,590,2005,5951
2,200500000003,5,1,21,1845,3,1,2,51,NaN,M,5054600,280000,590,2005,5951
3,200500000004,5,1,4,1615,1,1,1,82,NaN,M,5098700,240800,590,2005,5982
4,200500000005,5,1,10,1945,3,1,3,478,NaN,M,5096400,247500,590,2005,59478


In [419]:
# recoupage de l'heure sur 24

In [420]:
crtrtic['h24']=crtrtic['hrmn'].str[0:2]




In [421]:
crtrtic.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,atm,com,adr,gps,lat,long,dep,year,cp,h24
0,200500000001,5,1,12,1900,3,2,1,11,CD41B,M,5051500,294400,590,2005,5911,19
1,200500000002,5,1,21,1600,1,2,1,51,rue de Lille,M,5053700,280200,590,2005,5951,16
2,200500000003,5,1,21,1845,3,1,2,51,NaN,M,5054600,280000,590,2005,5951,18
3,200500000004,5,1,4,1615,1,1,1,82,NaN,M,5098700,240800,590,2005,5982,16
4,200500000005,5,1,10,1945,3,1,3,478,NaN,M,5096400,247500,590,2005,59478,19


In [ ]:
# Export vers fichiers de sortie 

In [ ]:
vehicules.to_csv('vehicules')
!cat vehicules.csv

In [ ]:
lieux.to_csv('lieux')
!cat lieux.csv

In [ ]:
usagers.to_csv('usagers')
!cat usagers.csv

In [426]:
crtrtic.to_json('caracteristiques')
!cat caracteristiques.csv

cat: crtrtic.json: Aucun fichier ou dossier de ce type
